# 準備

In [23]:
import gradio as gr
import getpass
import os

# get password from user
api_key = getpass.getpass("OpenAI API キーを入力してください: ")
os.environ["OPENAI_API_KEY"] = api_key

# Gradio の機能紹介

## 簡単なインターフェースの実装

In [4]:
def text2text(text):
    text = "<<" + text + ">>"
    return text

In [ ]:
input_text = gr.Text(label="入力")
output_text = gr.Text(label="出力")

demo = gr.Interface(inputs=input_text, outputs=output_text, fn=text2text)
demo.launch(debug=True)

## ブロックの実装

In [5]:
def text2text_rich(text):
    top = "^" * len(text)
    bottom = "v" * len(text)
    text = f" {top}\n<{text}>\n {bottom}"
    return text

In [ ]:
with gr.Blocks() as demo:
    input_text = gr.Text(label="入力")
    button1 = gr.Button(value="Normal")
    button2 = gr.Button(value="Rich")
    output_text = gr.Text(label="出力")

    button1.click(inputs=input_text, outputs=output_text, fn=text2text)
    button2.click(inputs=input_text, outputs=output_text, fn=text2text_rich)
demo.launch()

## 重要なコンポーネント

In [ ]:
def audio_upload(audio):
    return audio


with gr.Blocks() as demo:
    # Audio
    audio = gr.Audio(label="音声", type="filepath")
    audio_output = gr.Text(label="アップロードされたファイル")
    audio.upload(inputs=audio, outputs=audio_output, fn=audio_upload)
    # Checkbox
    checkbox = gr.Checkbox(label="チェックボックス")
    # File
    file = gr.File(label="ファイル", file_types=["image"])
    # Number
    number = gr.Number(label="数値")
    # Markdown
    markdown = gr.Markdown(label="Markdown", value="# タイトル\n## サブタイトル\n本文")
    # Slider
    slider = gr.Slider(
        label="スライダー", minimum=-10, maximum=10, step=0.5, interactive=True
    )
    # Textbox
    textbox = gr.Textbox(label="テキストボックス")

demo.launch()

## UI の工夫

In [ ]:
with gr.Blocks() as demo:
    # Accordion
    with gr.Accordion(label="アコーディオン"):
        gr.Text(value="アコーディオンの中身")
    with gr.Row():
        gr.Text(value="左")
        gr.Text(value="右")

    with gr.Row():
        with gr.Column():
            gr.Text(value="(0, 0)")
            gr.Text(value="(1, 0)")
        with gr.Column():
            gr.Text(value="(0, 1)")
            gr.Text(value="(1, 1)")

    with gr.Tab(label="タブ1"):
        gr.Text(value="コンテンツ1")
    with gr.Tab(label="タブ2"):
        gr.Text(value="コンテンツ2")

demo.launch()

In [ ]:
with gr.Blocks() as demo:
    # render
    slider = gr.Slider(label="個数", minimum=0, maximum=10, step=1)

    @gr.render(inputs=slider)
    def render_blocks(value):
        for i in range(value):
            gr.Text(value=f"Block {i}")


demo.launch()

## 状態を保持する

In [ ]:
with gr.Blocks() as demo:
    username = gr.State("")
    text_input = gr.Text(label="ユーザ名")
    button1 = gr.Button("決定")
    button2 = gr.Button("自分の名前を表示")
    text_output = gr.Text(label="出力")
    button1.click(inputs=text_input, outputs=username, fn=lambda x: x)
    button2.click(inputs=username, outputs=text_output, fn=lambda x: x)

demo.launch()

## チャット UI を作る

In [ ]:
from langchain_openai.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")


def history2messages(history):
    messages = []
    for user, assistant in history:
        messages.append({"role": "user", "content": user})
        messages.append({"role": "assistant", "content": assistant})
    return messages


def chat(message, history):
    messages = history2messages(history)
    messages.append({"role": "user", "content": message})
    response = llm.invoke(messages)
    return response.content


demo = gr.ChatInterface(chat)

demo.launch(debug=True)